# Reordering computations

This is an example of A-stationary/row-major (Gusstavson) spM-spM, where computations use coordinate-space reordering.

First, include some libraries

In [ ]:
# Begin - startup boilerplate code

import pkgutil

if 'fibertree_bootstrap' not in [pkg.name for pkg in pkgutil.iter_modules()]:
  !python3 -m pip  install git+https://github.com/Fibertree-project/fibertree-bootstrap --quiet

# End - startup boilerplate code


from fibertree_bootstrap import *
fibertree_bootstrap(style="tree", animation="movie")

In [ ]:
M = 6
K = 6
N = 6
seed = 20

a = Tensor.fromRandom(["M", "K"], [M, K], [0.7, 0.8], 5, seed=seed)
a.setColor("blue").setName("A")
displayTensor(a)

b = Tensor.fromRandom(["K", "N"], [K, N], [0.7, 0.65], 5, seed=seed)
b.setColor("green").setName("B")
displayTensor(b)



## A-stationary/row-major spMspM

No reordering

In [ ]:
z = Tensor(rank_ids=["M", "N"], shape=[M,N])
z.setName("Z")

a_m = a.getRoot()
b_k = b.getRoot()
z_m = z.getRoot()

canvas = createCanvas(a, b, z)

for m, (z_n, a_k) in z_m << a_m:
    for k, (a_val, b_n) in a_k & b_k:
        for n, (z_ref, b_val) in z_n << b_n:
            z_ref += a_val * b_val
            canvas.addFrame((m, k), (k, n), (m, n))

displayTensor(z)
displayCanvas(canvas)

## A-stationary/row-major spMspM

Random coordinate-based reordering of the M-rank of A

In [ ]:
import copy
from random import shuffle

z = Tensor(rank_ids=["M", "N"], shape=[M,N])
z.setName("Z")

a_m = a.getRoot()
b_k = b.getRoot()
z_m = z.getRoot()

canvas = createCanvas(a, b, z)

m_coords = [c for c in a_m.getCoords()]
shuffle(m_coords)

for m in m_coords:
    a_k = a_m.getPayload(m)
    z_n = z_m.getPayloadRef(m)
    
    for k, (a_val, b_n) in a_k & b_k:
        for n, (z_ref, b_val) in z_n << b_n:
            z_ref += a_val * b_val
            canvas.addFrame((m, k), (k, n), (m, n))

displayTensor(z)
displayCanvas(canvas)

## A-stationary/row-major spMspM

Random position-based reordering of the M-rank of A

In [ ]:
import copy
from random import shuffle

z = Tensor(rank_ids=["M", "N"], shape=[M,N])
z.setName("Z")

a_m = a.getRoot()
b_k = b.getRoot()
z_m = z.getRoot()

canvas = createCanvas(a, b, z)

m_positions = [n for n in range(len(a_m))]
shuffle(m_positions)

# Note: the following list is not a Fiber (coordinates not sorted)

for m, a_k in [a_m[pos] for pos in m_positions]:

    z_n = z_m.getPayloadRef(m)
    
    for k, (a_val, b_n) in a_k & b_k:
        for n, (z_ref, b_val) in z_n << b_n:
            z_ref += a_val * b_val
            canvas.addFrame((m, k), (k, n), (m, n))

displayTensor(z)
displayCanvas(canvas)